# Linear Program Formulations
Based on Lab 9 of ENGRI 1101 Fall 2019

<font color='red'> **Comments in red** </font>

<font color='blue'> **Solutions in blue** </font>

**Objectives:**
- Practice formulating linear programming problems
- Introduce the idea of linear relaxation
- Demonstrate the relationship between an integer program and its linear relaxation

**Brief description:**
   In this lab we will see examples of modeling various problems as Linear Programs (LPs), coding and solving them with OR-Tools, and start considering the question of what can be done when the desired output of a mathematical model should be integer.

In [1]:
# imports -- don't forget to run this cell!
from ortools.linear_solver import pywraplp as OR

## Part 1: A Work-scheduling problem
[Adapted from Hillier & Lieberman] A new company is being set up to take telephone orders. The phone lines are staffed around the clock. People are hired to work in 8-hour shifts, starting at 6AM, 8AM, 12PM, 4PM, or 10PM. From survey data, the management has concluded that there are minimum staffing requirements at certain times of the day, as given in the table below.

| Time period   | Minimum Staff Size |
|---------------|--------------------|
| 6AM-8AM       | 48                 |
| 8AM-10AM      | 79                 |
| 10AM-Noon     | 65                 |
| Noon-2PM      | 87                 |
| 2PM-4PM       | 64                 |
| 4PM-6PM       | 87                 |
| 6PM-8PM       | 64                 |
| 8PM-10PM      | 73                 |
| 10PM-Midnight | 82                 |
| Midnight-6AM  | 43                 |

A person is paid differently, depending on their assigned shift, where the daily salary is 170, 160, 175, 180, and 195, for the five shifts, respectively. The management would like to decide the number of people to be hired for each shift so as to minimize the total salary costs.
The first part of this lab is aimed at helping you to formulate this problem as a linear programming problem. As usual, the first step is to make a first attempt at stating the decision variables.

**Q1:** Let’s call the variables $x_1, x_2, x_3, x_4,$ and $x_5$. Explain what each of these variables represents.

**A:** <font color='blue'> See the table. </font>

| Variable      | Meaning                                |
|---------------|----------------------------------------|
| $x_1$         | # of people hired for 6AM - 2PM shift  |    
| $x_2$         | # of people hired for 8AM - 4PM shift  | 
| $x_3$         | # of people hired for 12PM - 8PM shift | 
| $x_4$         | # of people hired for 4PM - 12AM shift | 
| $x_5$         | # of people hired for 10PM - 6AM shift | 

**Q2:** In terms of these decision variables, express the objective function.

**A:** <font color='blue'> $\min [170x_1 + 160x_2 + 175x_3 + 180x_4 + 195x_5]$ </font>

**Q3:** In your own words (i.e., using sentences and not mathematical notation) briefly describe what conditions must be satisfied for a given assignment of values to your decision variables to correspond to a feasible solution.

**A:** <font color='blue'> The number of people assigned at a given time period (all of the people whose shift contains that time period) must meet the required number of people to be staffed during that time. </font>

**Q4:** Now try to express these restrictions as linear constraints in your decision variables. As a start, consider the period from 6AM to 8AM. You are required to staff this period with at least 48 people. Which shifts work during this time? How should you write this as a constraint?

**A:** <font color='blue'> The only shift that works at this time is the 6AM - 2PM shift represented by $x_1$. Hence, the constraint is $$x_1 \geq 48$$  </font>

**Q5:** Now complete your linear programming formulation (write the rest of the constraints).

**A:** <font color='blue'> See the table. </font>

|                           |
|---------------------------|
| $x_1 \geq 48$             |
| $x_1 + x_2 \geq 79$       |
| $x_1 + x_2 \geq 65$       |
| $x_1 + x_2 + x_3 \geq 87$ |
| $x_2 + x_3 \geq 64$       |
| $x_3 + x_4 \geq 87$       |
| $x_3 + x_4 \geq 64$       |
| $x_4 \geq 73$             |
| $x_4 + x_5 \geq 82$       |
| $x_5 \geq 43$             |

**Q6:** If deleting a constraint from an LP does not create any new feasible solutions, then the constraint is called *redundant*. Why is your constraint for the 10PM-midnight shift redundant? Feel free to cross it out.

**A:** <font color='blue'> Because we already have the constraint that $x_4 \geq 73$ and $x_5 \geq 43$. It follows that $x_4 + x_5 \geq 116$. Hence, $x_4 + x_5 \geq 82$ is always satisifed and does not remove any feasible solutions. </font>

Let’s look at the model for this problem in OR-Tools.

In [2]:
# a work schedule model
def WorkSched(integer = False):
    # define the model
    m = OR.Solver('work_sched', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # decision variables
    if integer:   
        x = {i: m.IntVar(0, m.infinity(), 'x_'+str(i)) for i in range(1,6)}
    else:
        x = {i: m.NumVar(0, m.infinity(), 'x_'+str(i)) for i in range(1,6)}

    # objective function
    m.Minimize(170*x[1] + 160*x[2] + 175*x[3] + 180*x[4] + 195*x[5])

    # constraints
    m.Add(x[1]                             >= 48)
    m.Add(x[1] + x[2]                      >= 79)
    m.Add(x[1] + x[2]                      >= 65)
    m.Add(x[1] + x[2] + x[3]               >= 87)
    m.Add(       x[2] + x[3]               >= 64)
    m.Add(              x[3] + x[4]        >= 87)
    m.Add(              x[3] + x[4]        >= 64) 
    m.Add(                     x[4]        >= 73)
    m.Add(                     x[4] + x[5] >= 82) 
    m.Add(                            x[5] >= 43)
    
    return m, x

In [3]:
# function to solve the model
def solve(model):
    model.Solve()
    print('Objective =', model.Objective().Value())
    print('Solution:')
    for v in model.variables():
        print(v.name(),':', v.solution_value())

Study the model and ensure that you understand how it corresponds to the model you came up with.

**Q7:** Did you write down the correct formulation? If not, what did you do wrong?

**A:** <font color='blue'> Depends. </font>

Now you will solve this model using OR-Tools. Run the cell below.

In [4]:
m, x = WorkSched()
solve(m)

Objective = 40135.0
Solution:
x_1 : 48.0
x_2 : 50.0
x_3 : 14.0
x_4 : 73.0
x_5 : 43.0


**Q8:** What is the objective value of the optimal solution? What is the optimal solution (i.e., the values of the decision variables)?

**A:**  <font color='blue'> The objective value is 40,135 and the solution is $x = (48, 50, 14, 73, 43)$ </font>

Each staff member is also expected to take care of processing a number of mail-in orders when not on the telephone. Each person in the 6am, 8am, noon, 4pm and 10pm shifts are expected to handle 30, 10, 10, 30, and 50 orders, respectively. The staffing is required to be sufficient to handle 6500 orders. 

**Q9:** Add a constraint to the model to account for the mail-in orders in the cell below.

In [5]:
m, x = WorkSched()
# TODO: Add your constraint here
# m.Add()

# SOLUTION
m.Add(30*x[1] + 10*x[2] + 10*x[3] + 30*x[4] + 50*x[5] >= 6500)

solve(m)

Objective = 40447.0
Solution:
x_1 : 48.0
x_2 : 49.999999999999986
x_3 : 14.0
x_4 : 73.0
x_5 : 44.60000000000001


**Q10:** Report the new solution below.

**A:**  <font color='blue'> The objective value is 40,447 and the solution is $x = (48, 50, 14, 73, 44.6)$ </font>

**Q11:** The solution to this model is not actually viable as a real-world hiring policy. Why not? Can you tweak the solution you found to give a viable (but not necessarily optimal) hiring plan? What is the total cost of your plan? 

**A:**  <font color='blue'> We can not hire $\frac{6}{10}$ of a person so this solution is not viable. Instead, we can round up and let $x_5 = 45$ to give a feasible solution with a total cost of 40,525. </font>

In effect, the issue in the previous question was that we have an integer programming (IP) problem, not a linear programming (LP) problem. Let's solve the same model with an integrality constraint.

In [6]:
m, x = WorkSched(integer=True)
# TODO: Add your mail-in order constraint from Q9

# SOLUTION
m.Add(30*x[1] + 10*x[2] + 10*x[3] + 30*x[4] + 50*x[5] >= 6500)

solve(m)

Objective = 40495.0
Solution:
x_1 : 48.0
x_2 : 51.0
x_3 : 13.0
x_4 : 74.0
x_5 : 44.0


**Q12:** Write down the optimal solution and its value below. How does the optimal value of the integer program compare to the optimal value of the linear programming problem, its so-called linear relaxation? How does the optimal IP solution compare with the solution you proposed above?

**A:**  <font color='blue'> The optimal solution is x = (48, 51, 13, 74, 44) with objective value 40,495 which is greater than it's linear program relaxation but less than the proprosed solution from rounding up.</font>

## Part 2: A Production Process Model
[from Winston] Sunco Oil has three different processes that can be used to manufacture various types of gasoline. Each process involves blending oils in the company’s catalytic cracker. 
- Running process 1 for an hour costs 5 dollars and requires 2 barrels of crude oil 1 and 3 barrels of crude oil 2. The output of running process 1 for an hour is 2 barrels of gas 1 and 1 barrel of gas 2. 
- Running process 2 for an hour costs 4 dollars and requires 1 barrel of crude 1 and 3 barrels of crude 2. The output from running process 2 for an hour is 3 barrels of gas 2. 
- Running process 3 for an hour costs 1 dollar and requires 2 barrels of crude 2 and 3 barrels of gas 2. The output from running process 3 for an hour is 2 barrels of gas 3. 

Each week, 200 barrels of crude 1, at 2 dollars/barrel, and 300 barrels of crude 2, at 3 dollars/barrel, may be purchased. All gas produced can be sold at the following per-barrel prices: gas 1, 9; gas 2, 10; gas 3, 24. The catalytic cracker can be used for only 95 hours each week. Sunco Oil wishes to devise a weekly production plan that maximizes the production’s profit (revenues minus costs).

**Q13:** The above paragraph is pretty dense. Draw a pictorial representation of the flow of materials in this problem: from crude oil, to processes, to barrels of gas for sale. It might help to have one picture for each process (e.g. one picture for process 1 indicating what materials it needs to run and what materials it spits out). You might also want a chart indicating how much each barrel of crude oil / each process costs, as well as how much we profit from each gas.

**A:** <font color='blue'> Something like the following diagram:</font>

![title](images-key/process_model_diagram.png)

You’ll spend the rest of lab thinking about how to formulate this problem as a linear program. There are many ways to do this, but the most straightforward is to use 8 decision variables. Three of these are $\text{proc}_1$,$\text{proc}_2$ and $\text{proc}_3$, the numbers of hours spent running processes 1, 2 and 3. 

**Q14:** You should also have decision variables $\text{crude}_1$, $\text{crude}_2$, $\text{gas}_1$, $\text{gas}_2$, and $\text{gas}_3$. What do $\text{crude}_i$ and $\text{gas}_j$ represent?

**A:** <font color='blue'> $\text{Crude}_i$ represents the number of barrels of crude oil $i$ purchased and $\text{gas}_j$ represents the number of barrels of gas $j$ manufactured. </font>

**Q15:** State the objective function in terms of these variables: be sure to account for the cost of running processes and buying crude oil, as well as the profit from selling gas.

**A:** <font color='blue'> $\max [9\text{gas}_1 + 10\text{gas}_2 + 24\text{gas}_3] - [2\text{crude}_1 + 3\text{crude}_2 + 5\text{proc}_1 + 4\text{proc}_2 + 1\text{proc}_3]$ </font>

In this problem, the primary decision variables are the $\text{proc}_i$ variables. We’ll need constraints that enforce that the other decision variables (the $\text{crude}_i$ and $\text{gas}_j$) line up with how much of each process we use.

**Q16:** For example, if we use two hours of process 1, five hours of process 2, and two hours of process 3, how much of each crude oil did we use? How much of each gas do we have to sell at the very end? (Hint: Be careful about the amount of gas 2 we have to sell at the end, since some of it will be used up in process 3.)

**A:** <font color='blue'> 
$$\text{crude}_1 = 2(2) + 1(5) = 9$$
$$\text{crude}_2 = 3(2) + 3(5) = 2(2) = 25$$
$$\text{gas}_1 = 2(2) = 4$$
$$\text{gas}_2 = 1(2) + 3(5) - 3(2) = 11$$
$$\text{gas}_3 = 2(2) = 4$$
</font>

**Q17:** Based on the previous question, why does a constraint like the following make sense:
$$\text{crude}_1 = 2\text{proc}_1 + \text{proc}_2$$

**A:** <font color='blue'> Because 2 barrels of crude 1 are used in process 1 and 1 barrel is used in process 2.</font>

**Q18:** Write four more equality constraints like the one above, one for each of $\text{crude}_2$, $\text{gas}_1$, $\text{gas}_2$, and $\text{gas}_3$ These constraints enforce that these extra decision variables represent what we want them two, based solely on how much time we spend on each process. (Hint: Your constraint for $\text{gas}_2$ will be a little tricky. You want $\text{gas}_2$ to represent the total amount of gas 2 that you have at the end of all processes, since this is what you sell for profit. However, some of gas 2 might be used up by process 3. Hence your gas 2 constraint will have three terms: two indicating how much you get from processes 1 and 2, and one indicating how much you lose from process 3.)

**A:** <font color='blue'>
$$\text{crude}_2 = 3\text{proc}_1 + 3\text{proc}_1 + 2\text{proc}_1$$
$$\text{gas}_1 = 2\text{proc}_1$$
$$\text{gas}_2 = 1\text{proc}_1 + 3\text{proc}_2 - 3\text{proc}_3$$
$$\text{gas}_3 = 2\text{proc}_3$$    
</font>

**Q19:** In addition to the five constraints written so far, we need 11 more: 8 of these are freebies (the non-negativity constraints on each of the eight decision variables). What other three constraints do we need? (Hint: what limits do we have from the problem statement?)

**A:** <font color='blue'>
$$\text{crude}_1 \leq 200$$
$$\text{crude}_2 \leq 300$$
$$\text{proc}_1 + \text{proc}_2 + \text{proc}_3 \leq 95$$
</font>

**Q20:** Finish writing the model below and then solve it! As a sanity check to both your code and model, the optimal solution should leave you with a cost of 1,425 dollars and involve spending all time on process 2.

In [7]:
# define the model
mod = OR.Solver('production_process', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# decision variables
proc = {i: mod.NumVar(0, mod.infinity(), 'proc_'+str(i)) for i in range(1,4)}
crude = {i: mod.NumVar(0, mod.infinity(), 'crude_'+str(i)) for i in range(1,3)}
gas = {i: mod.NumVar(0, mod.infinity(), 'gas_'+str(i)) for i in range(1,4)}

# objective function
# TODO: Add the objective function
# mod.Maximize()

# SOLUTION
mod.Maximize((9*gas[1] + 10*gas[2] + 24*gas[3])     # revenue from selling
             - (2*crude[1] + 3*crude[2])            # cost of crude oil
             - (5*proc[1] + 4*proc[2] + 1*proc[3])) # cost of running processes

# constraints
# TODO: Add the constraints. 
# Hint: the syntax is mod.Add(some expression)
# Hint: for equality constraints, use ==

# SOLUTION

# enforce the extra decison variables to represent what we want
mod.Add(crude[1] == 2*proc[1] + proc[2])
mod.Add(crude[2] == 3*proc[1] + 3*proc[2] + 2*proc[3])
mod.Add(gas[1] == 2*proc[1])
mod.Add(gas[2] == 1*proc[1] + 3*proc[2] - 3*proc[3])
mod.Add(gas[3] == 2*proc[3])

# limit of oil that can bee purchased
mod.Add(crude[1] <= 200)
mod.Add(crude[2] <= 300)

# time limit
mod.Add(proc[1] + proc[2] + proc[3] <= 95);

Now solve it!

In [8]:
# solve and print solution
mod.Solve()
print('Objective =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

Objective = 1424.9999999999998
Solution:
proc_1 : 0.0
proc_2 : 95.0
proc_3 : 0.0
crude_1 : 95.00000000000001
crude_2 : 285.00000000000006
gas_1 : 0.0
gas_2 : 285.0
gas_3 : 0.0
